# Segmented Body Part Enlarger

# Install Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install imagemagick
!pip install Wand

In [ ]:
!pip install -U torch==2.2.1 torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.checkpoint import Checkpointer
import os
import random

# Run Segmentation Predictor

In [ ]:
path = "/content/drive/MyDrive/ML_Stuff/model_9"
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.MODEL.WEIGHTS = os.path.join(path, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

[05/04 00:51:22 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/ML_Stuff/model_9/model_final.pth ...


In [ ]:
from detectron2.utils.visualizer import ColorMode

im = cv2.imread('test.jpg')
outputs = predictor(im)

v = Visualizer(im[:, :, ::-1],
  scale=1.0,
)

instances = outputs["instances"].to("cpu")

if len(instances) > 0:
  highest_score_index = instances.scores.argmax()
  highest_score_instance = instances[highest_score_index:highest_score_index + 1]
  mask_res = highest_score_instance.pred_masks[0]
  box = highest_score_instance.pred_boxes
else:
  highest_score_instance = instances
  mask_res = None

v = v.draw_instance_predictions(highest_score_instance)
cv2_imshow(v.get_image()[:, :, ::-1])

Mask and ROI Preparation

In [36]:
import cv2
import numpy as np
from wand.image import Image

mask_np = mask_res.numpy().astype(np.uint8)

# Binary mask for layer masking later
mask = mask_np * 255

# The isolated region on the real image that we want to transform
isolated_roi = cv2.bitwise_and(im, im, mask=mask)

# Segmented Body Part Enlarging

Distortion

In [ ]:
from PIL import Image, ImageFilter

def apply_perspective_warp(image, roi_corners, transformed_corners):
  src_points = np.array(roi_corners, dtype=np.float32)
  dst_points = np.array(transformed_corners, dtype=np.float32)

  M = cv2.getPerspectiveTransform(src_points, dst_points)

  warped_roi = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))

  return warped_roi

def get_warp_factor(bbox, img_width, img_height):
  h_factor = 400
  v_factor = 200
  x1, y1, x2, y2 = bbox

  distance_to_left = x1
  distance_to_right = img_width - x2
  distance_to_top = y1
  distance_to_bottom = img_height - y2

  h_stretch = h_factor * (img_width / (distance_to_left + distance_to_right))
  v_stretch = v_factor * (img_height / (distance_to_top + distance_to_bottom))

  return int(h_stretch), int(v_stretch)

x1, y1, x2, y2 = box.tensor.numpy()[0].astype('int')
center_x = (x1 + x2) // 2
center_y = (y1 + y2) // 2
height, width = isolated_roi.shape[:2]
h_stretch, v_stretch = get_warp_factor((x1, y1, x2, y2), width, height)

print("Center X: " + str(center_x) + ", Center Y: " + str(center_y))
print("Height: " + str(height) +", Width: " + str(width))
print("Top Left: " + str(x1) + ", Top Right: " + str(x2) + ", Bottom Left: " + str(y1) + ", Bottom Right: " + str(y2))
print("Horizontal Stretch: " + str(h_stretch) + ", Vertical Stretch: " + str(v_stretch))

roi_corners = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
transformed_corners = [(x1, y1), (x2, y1), (x2 + h_stretch, y2 + v_stretch), (x1 - h_stretch, y2 + v_stretch)]

transformed_roi = apply_perspective_warp(isolated_roi.copy(), roi_corners, transformed_corners)
transformed_roi_img = Image.fromarray(transformed_roi).filter(ImageFilter.ModeFilter(size=10))
transformed_roi = np.array(transformed_roi_img)

cv2_imshow(transformed_roi)

In [ ]:
# Transformed binary mask
image_mask = cv2.cvtColor(transformed_roi, cv2.COLOR_BGR2GRAY)
_, image_mask = cv2.threshold(image_mask, 1, 255, cv2.THRESH_BINARY)

masked_roi = cv2.bitwise_and(transformed_roi, transformed_roi, mask=image_mask)
mask_inv = cv2.bitwise_not(image_mask)
masked_original = cv2.bitwise_and(im, im, mask=mask_inv)

combined_image = cv2.add(masked_original, masked_roi)
cv2_imshow(combined_image)